In [12]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup

from PIL import Image
import io

import time

**ブラウザの起動**

**各ページのURLを取得する関数**
1. SUUMOの物件情報(indexページ)は、基本的に内部リンク（'https:~がない"）形式でHTMLが作成されているため、内部リンクのみ全体のページから取得する
2. 取得したURLをリスト形式にまとめる
3. URLのリストを返す関数を作成する

In [2]:
def get_urls(soup) -> list:
    """
    一覧のページからURL個別ページのURLを取得する関数
    """
    h2_elems = soup.find_all('h2', attrs={'class' : 'property_unit-title'})
    
    urls = []
    
    for h2_elem in h2_elems:
        a_elem = h2_elem.find('a')
        url = a_elem.attrs['href']
        
        urls.append(url)
        
    return urls

**各ページの物件情報を収集する関数**
- 入力は各ページの内部リンクのURL（シングル）
- 出力は各ページのsoup

In [3]:
def get_page_soup(internal_url:str):
    """
    各物件へのリンクを作成して、アクセスして、各ページのsoupを出力する
    """
    # ページ内リンクをドメインと結びつけて直にアクセスできるリンクに変換している
    page_url = 'https://suumo.jp' + internal_url
    page_res = requests.get(page_url)
    page_soup = BeautifulSoup(page_res.content, 'html.parser')
    
    return page_soup

**各ページの物件情報を収集する関数**
- 入力は各ページのsoup
- 出力は各ページの物件情報が格納された辞書

In [4]:
def get_house_details(page_soup) -> dict:
    """[summary]　
    　その後に各物件のページから「物件情報」にアクセスして、「物件情報」のテーブルまるごと取得する。
    Args
        internal_url[str]:SUUMOの各物件の内部リンク
    """
    #物件詳細のページへのリンクの取得
    house_details_a_elem = page_soup.find('a', attrs={'class' : 'tabOutline2'})
    
    if house_details_a_elem is None:
        house_details_a_elem = page_soup.find('a', attrs={'class' : 'tabOutline'})
        
    house_details_url = house_details_a_elem.attrs['href']
    
    #物件詳細のページへのアクセス    
    house_details_res = requests.get(house_details_url)
    house_details_soup = BeautifulSoup(house_details_res.content, 'html.parser')
    
    # テーブルの取得
    house_details_info = house_details_soup.find('table', {'class': 'pCell10'})

    return house_details_info

**各ページの物件情報を収集する関数**
- 入力はtable要素
- 出力はth要素がKey、td要素がValueになった辞書

In [5]:
def extract_table_data(table) -> dict:
    """
    　tableのHTMLからthとtd内の情報を取得する　
    """
    house_details_dict = {}

    for row in table.find_all('tr'):
        data_head = row.find_all('th')
        data_value = row.find_all('td')

        data_head_01 = data_head[0].text.replace('\n', '').replace('ヒント', '')
        data_value_01 = data_value[0].text.replace('\n', '').replace('\r', '').replace('\t', '')

        try:
            if data_head[1] is not None:
                data_head_02 = data_head[1].text.replace('\n', '').replace('ヒント', '')
                data_value_02 = data_value[1].text.replace('\n', '').replace('\r', '').replace('\t', '').replace('\u3000', '').replace('\xa0', '').replace('[乗り換え案内]', '')

                house_details_dict[data_head_01] = data_value_01
                house_details_dict[data_head_02] = data_value_02
        except IndexError:
            house_details_dict[data_head_01] = data_value_01
            
    return house_details_dict

**各物件のタイトルとコメントを取得する関数**

In [6]:
def get_title_and_comment(page_soup):
    """
    サイト上部にあるタイトルとコメントを取得する関数
    """
    try:
        house_title = page_soup.find('h2', attrs = {'class' : 'fs16'})
        house_comment = page_soup.find('p', attrs = {'class' : 'fs14'})
        house_dict = {'title':house_title.text, 'comment':house_comment.text}
    except:
        house_dict = {'title':'', 'comment':''}
    return house_dict

## テスト用のコードTips

In [7]:
# page_soup = get_page_soup(urls[1]) 
# table = get_house_details(page_soup)
# house_info_dict = extract_table_data(table)
# house_text_dict = get_title_and_comment(page_soup)
# get_house_img(page_soup, 1)
# urls

**家の画像の取得**
- 画像の名前を一意に決めるためにhouse_idを引数に入れている

In [8]:
def get_house_img(page_soup, house_id):
    """
    各ページの写真を取得して、写真をHouseId_IMGIDの形式で保存する。
    """
    imgs = page_soup.find_all('img')
    img_list = list()
    img_id = 0
    # 以下で始まるのはIMGタグだがアクセスできないため排除する
    un_img_signal = 'img01.suumo.com'
    
    for img in imgs:
        try:
            img_url = img['rel']
            img_tag = img['alt']
        except KeyError:
            img_url = un_img_signal
            img_tag = ""

        if not un_img_signal in img_url:
            img_name = str(house_id) + '_' + str(img_id)
            img_id += 1
            img = Image.open(io.BytesIO(requests.get(img_url).content))
            img.save(f'imgs/{img_name}.jpg')
            img_list.append({'house_id':house_id, 'img_id':img_id, 'img_tag':img_tag, 'img_name':img_name})
    
    return img_list

**Index1ページ分のURL**

- ここのループでは、Index1ページ分のURLをすべて取ってきている

In [9]:
def get_index_info(urls:list, house_info:list, house_id:int) -> list:
    for url in urls:
        page_soup = get_page_soup(url)
        table = get_house_details(page_soup)
        house_info_dict = extract_table_data(table)
        house_text_dict = get_title_and_comment(page_soup)
        house_img_list = get_house_img(page_soup, house_id)

        house_dict = {'House_ID': house_id, 'text':house_text_dict, 'info':house_info_dict, 'imgs':house_img_list}
        house_info.append(house_dict)
        
        house_id += 1
        
        time.sleep(20)
        
    return house_info, house_id

In [10]:
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
url = 'https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999'
browser.get(url)

count = 0

url = browser.current_url
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
a_elems = soup.find_all('div', attrs={'class' : 'property_unit-header'})



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


**すべてのページを見れるように次へのボタンを押す**

- このままでは、すべてのページを自動的にクロールすることは不可能。そこで、すべてのページをクロールできるようにプログラムで次へのボタンを押すことを試みます。

In [ ]:
data_list = []
#house_info = []
house_id = 0

count = 0

url = 'https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999'
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
browser.get(url)

while True:
    url = browser.current_url
    print(url)
    #ブラウザの起動
    options = Options()
    options.add_argument('--headless')
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    browser.get(url)
    
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        #a_elems = soup.find_all('div', attrs={'class' : 'property_unit-header'})
        urls = get_urls(soup)
        house_info, house_id = get_index_info(urls, data_list, house_id)
        
        browser.find_element_by_link_text('次へ').click()
        if count % 10 == 0:
            print("pages:{0}".format(count))
            print('==============================================')
            time.sleep(60)
        time.sleep(60)
        
        count += 1
            
    except:
        browser.quit()
        break
    



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999


In [ ]:
house_info